## Question 1
Find the 10 most frequent adjective-noun pairs in adj_noun_pairs.txt. The code skeleton is given below and you can only use RDD operations.

Data can be downloaded at http://www.cse.ust.hk/msbd5003/data/adj_noun_pairs.txt



In [38]:
numPartitions = 10
lines = sc.textFile('./data/adj_noun_pairs.txt', numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

#FILL IN YOUR CODE HERE
pair_freqs = pairs.map(lambda p: (p,1)).reduceByKey(lambda f1, f2: f1 + f2) 
                #  .filter(lambda pf: pf[1] >= 100)
pair_freqs.sortBy(lambda x: x[1], False).take(10)

[((u'external', u'link'), 8136), ((u'19th', u'century'), 2869), ((u'20th', u'century'), 2816), ((u'same', u'time'), 2744), ((u'first', u'time'), 2632), ((u'civil', u'war'), 2236), ((u'large', u'number'), 2094), ((u'other', u'hand'), 2043), ((u'political', u'party'), 1899), ((u'other', u'country'), 1857)]

In [39]:
#sample
result = pairs.map(lambda (a,n): (a+' '+n, 1)).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
print result.take(10)

[(u'external link', 8136), (u'19th century', 2869), (u'20th century', 2816), (u'same time', 2744), (u'first time', 2632), (u'civil war', 2236), (u'large number', 2094), (u'other hand', 2043), (u'political party', 1899), (u'other country', 1857)]


## Question 2

In the example relational database used in the lectures, each customer is served by a salesperson, which is recorded in the "SalesPerson" column in the "Customer" table (see the database schema below).

Use DataFrame API to generate a report of each salesperson's performance, which is the total net price of his/her customer's orders.  Note that the an order consists of multiple order details, and the net price of an order detail is UnitPrice * OrderQty * (1 - UnitPriceDiscount).

All salespeople should be reported, even if they have 0 order.

Data can be downloaded here:

http://www.cse.ust.hk/msbd5003/data/Customer.csv

http://www.cse.ust.hk/msbd5003/data/SalesOrderDetail.csv

http://www.cse.ust.hk/msbd5003/data/SalesOrderHeader.csv

In [33]:
dfCustomer = spark.read.csv('./data/Customer.csv', header=True, inferSchema=True)
dfDetail = spark.read.csv('./data/SalesOrderDetail.csv', header=True, inferSchema=True)
dfHeader = spark.read.csv('./data/SalesOrderHeader.csv', header=True, inferSchema=True)

#FILL IN YOUR CODE HERE

In [34]:
dfCustomer.printSchema()
dfDetail.printSchema()
dfHeader.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- NameStyle: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Suffix: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- SalesPerson: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- PasswordHash: string (nullable = true)
 |-- PasswordSalt: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)

root
 |-- SalesOrderID: integer (nullable = true)
 |-- SalesOrderDetailID: integer (nullable = true)
 |-- OrderQty: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- UnitPriceDiscount: double (nullable = true)
 |-- LineTotal: double (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDat

In [36]:
#join

dfsale = dfCustomer.join(dfHeader, 'CustomerID','left_outer') \
                   .select('CustomerID', 'SalesPerson', 'SalesOrderID')
dffinal = dfsale.join(dfDetail, 'SalesOrderID','left_outer') \
                .select('CustomerID', 'SalesPerson', 'SalesOrderID', 'UnitPrice', 'OrderQty', 'UnitPriceDiscount')
result = dffinal.select('*', (dffinal.UnitPrice * dffinal. OrderQty
                      * (1 - dffinal.UnitPriceDiscount)).alias('netprice')) \
       .groupby('SalesPerson').sum('netprice') \
       .withColumnRenamed('sum(netprice)', 'TotalNetPrice').na.fill(0)
result.orderBy('TotalNetPrice', ascending=False).show(truncate = False)

+------------------------+------------------+
|SalesPerson             |TotalNetPrice     |
+------------------------+------------------+
|adventure-works\jae0    |425979.1086639994 |
|adventure-works\linda3  |170064.83797000017|
|adventure-works\shu0    |112646.20642400002|
|adventure-works\david8  |0.0               |
|adventure-works\pamela0 |0.0               |
|adventure-works\michael9|0.0               |
|adventure-works\garrett1|0.0               |
|adventure-works\josé1   |0.0               |
|adventure-works\jillian0|0.0               |
+------------------------+------------------+



In [35]:
# sample solution
#Calculate net price for each order
orders = dfDetail.select('*', (dfDetail.UnitPrice * dfDetail.OrderQty
 * (1 - dfDetail.UnitPriceDiscount)).alias('netprice'))\
 .groupBy('SalesOrderID').sum('netprice') \
 .withColumnRenamed('sum(netprice)', 'TotalPrice')
#Three tables join
result = dfCustomer.join(dfHeader, 'CustomerID','left_outer').join(orders,'SalesOrderID','left_outer').groupBy('salesPerson').sum('TotalPrice').na.fill(0)
result.orderBy('sum(TotalPrice)', ascending=False).show(truncate = False)

+------------------------+------------------+
|salesPerson             |sum(TotalPrice)   |
+------------------------+------------------+
|adventure-works\jae0    |425979.108664     |
|adventure-works\linda3  |170064.83797000002|
|adventure-works\shu0    |112646.20642399997|
|adventure-works\pamela0 |0.0               |
|adventure-works\jillian0|0.0               |
|adventure-works\david8  |0.0               |
|adventure-works\garrett1|0.0               |
|adventure-works\michael9|0.0               |
|adventure-works\josé1   |0.0               |
+------------------------+------------------+



##  Question 3
In social networks, "zombies" are users that follow many other users, but are not followed by anybody. Given the following graph representing a social network, find the name of the largest zombie, i.e., the user who follows the most other users but is not followed by anybody. 



In [1]:
sc.addPyFile("/Users/allenwong/graphframes-0.5.0-spark2.1-s_2.11.jar")

In [23]:
from graphframes import *

# Vertices DataFrame
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("c", "a", "friend"),
 ("b", "c", "follow"),
 ("d", "a", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("f", "b", "follow"),
 ("c", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "d", "friend"),
 ("g", "c", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

# FILL IN YOUR CODE HERE

In [3]:
g.triplets.show()

+--------------+------------+--------------+
|           src|        edge|           dst|
+--------------+------------+--------------+
|  [g,Gabby,60]|[g,d,friend]|  [d,David,29]|
|  [f,Fanny,38]|[f,d,follow]|  [d,David,29]|
| [e,Esther,32]|[e,d,follow]|  [d,David,29]|
|[c,Charlie,37]|[c,d,follow]|  [d,David,29]|
|  [g,Gabby,60]|[g,c,friend]|[c,Charlie,37]|
|  [f,Fanny,38]|[f,c,follow]|[c,Charlie,37]|
|    [b,Bob,36]|[b,c,follow]|[c,Charlie,37]|
|  [f,Fanny,38]|[f,b,follow]|    [b,Bob,36]|
|  [a,Alice,34]|[a,b,follow]|    [b,Bob,36]|
|  [g,Gabby,60]|[g,a,friend]|  [a,Alice,34]|
| [e,Esther,32]|[e,a,follow]|  [a,Alice,34]|
|  [d,David,29]|[d,a,follow]|  [a,Alice,34]|
|[c,Charlie,37]|[c,a,friend]|  [a,Alice,34]|
+--------------+------------+--------------+



In [49]:
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  d|       4|
|  c|       3|
|  b|       2|
|  a|       4|
+---+--------+



In [70]:
g.outDegrees.agg({"outDegree": "max"}).show()

+--------------+
|max(outDegree)|
+--------------+
|             3|
+--------------+



In [62]:
g.vertices.join(g.outDegrees, 'id') \
          .join(g.inDegrees, 'id', 'left_outer').filter('inDegree is null').show()

+---+------+---+---------+--------+
| id|  name|age|outDegree|inDegree|
+---+------+---+---------+--------+
|  g| Gabby| 60|        3|    null|
|  f| Fanny| 38|        3|    null|
|  e|Esther| 32|        2|    null|
+---+------+---+---------+--------+



In [31]:
# sample solution
e1 = g.edges.filter("relationship = 'follow'")
v1 = e1.groupBy('dst').count().select('dst')
v2 = e1.groupBy('src').count().orderBy('count', ascending=False)
v3 = v2.select('src').subtract(v1.select('dst'))
v4 = v3.join(v2,'src').first()
v.filter(v['id']==v4.src).show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  f|Fanny| 38|
+---+-----+---+



In [30]:
v4.show()

+---+-----+
|src|count|
+---+-----+
|  f|    3|
|  e|    2|
+---+-----+

